# Setup

In [1]:
!source /etc/profile.d/modules.sh
!module load CUDA/11.2
!export PATH=/local/java/cuda-11.2/bin:$PATH
!export LD_LIBRARY_PATH=/local/java/cuda-11.2/lib64:/local/java/cudnn-8.1_for_cuda_11.2/lib64:$LD_LIBRARY_PATH  # this line is needed for it to recognise gpu devices -- run this in the terminal
!export CUDA_HOME=/local/java/cuda-11.2
import os 
import tensorflow as tf
import numpy as np
import random
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten, Dropout, GlobalAveragePooling2D, Lambda
from keras import backend as K

print(tf.__version__)  # 2.12.0
print(tf.config.list_physical_devices('GPU'))  # should show gpu available


2025-02-10 14:12:11.859689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 14:12:11.947416: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 14:12:11.969822: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 14:12:13.507137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-02-10 14:12:17.698084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-10 14:12:17.706480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-10 14:12:17.706579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Prepare the image pairs

In [2]:
TRAIN_PATH = "train"

def create_pairs():
    pairs = []
    labels = [] 

    for category in os.listdir(TRAIN_PATH):
        anchor_dir = os.path.join(TRAIN_PATH, category, "anchor")
        positive_dir = os.path.join(TRAIN_PATH, category, "positive")
        negative_dir = os.path.join(TRAIN_PATH, category, "negative")

        anchor_list = [os.path.join(anchor_dir, f) for f in os.listdir(anchor_dir)]
        positive_list = [os.path.join(positive_dir, f) for f in os.listdir(positive_dir)]
        negative_list = [os.path.join(negative_dir, f) for f in os.listdir(negative_dir)]
        
        for a in anchor_list:
            for p in positive_list:
                pairs.append((a,p))
                labels.append(1)
            for n in negative_list:
                pairs.append((a,n))
                labels.append(0)
    
    # shuffle the dataset 
    combined = list(zip(pairs, labels))
    random.shuffle(combined)
    pairs, labels = zip(*combined)
    
    return np.array(pairs), np.array(labels)

pairs, labels = create_pairs()


# Preprocess the images

In [3]:
def preprocess(image_path):
    # read the file 
    raw = tf.io.read_file(image_path)
    img = tf.io.decode_image(raw, expand_animations=False, channels = 3)
    img = tf.image.resize(img, size = (224, 224), preserve_aspect_ratio=True)
    img = tf.image.resize_with_crop_or_pad(img, 224, 224)
    img = tf.cast(img, tf.float32)/255.0
    return img 

def preprocess_pair(pair):
    imgA = preprocess(pair[0])
    imgB = preprocess(pair[1])
    return (imgA, imgB)

# Hyper parameters

In [4]:
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE_TRAIN = 8
BATCH_SIZE_VALIDATION = 2
MARGIN = 1.0

# Build the training and validation dataset

In [5]:
# split data into training and validation dataset
train_pairs, val_pairs, train_labels, val_labels = train_test_split(
    pairs, labels, test_size=0.3, random_state=42
)

In [6]:
def build_training_dataset():
    pairs_tensor = tf.convert_to_tensor(train_pairs)
    labels_tensor = tf.convert_to_tensor(train_labels)
    result = tf.data.Dataset.from_tensor_slices((pairs_tensor, labels_tensor))

    # preprocess the images 
    result = result.map(lambda pair, label: (preprocess_pair(pair), label))
    result = result.batch(BATCH_SIZE_TRAIN)
    result = result.prefetch(tf.data.AUTOTUNE)

    return result 

def build_validation_dataset():
    pairs_tensor = tf.convert_to_tensor(val_pairs)
    labels_tensor = tf.convert_to_tensor(val_labels)
    result = tf.data.Dataset.from_tensor_slices((pairs_tensor, labels_tensor))

    # preprocess the images 
    result = result.map(lambda pair, label: (preprocess_pair(pair), label))
    result = result.batch(BATCH_SIZE_VALIDATION)
    result = result.prefetch(tf.data.AUTOTUNE)

    return result 

train_dataset = build_training_dataset()
validation_dataset = build_validation_dataset()

    

2025-02-10 14:12:17.800181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 14:12:17.801787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-10 14:12:17.802200: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-10 14:12:17.802392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

# Calculate euclidean distance

In [7]:
def euclidean_distance(vectors):
    x, y = vectors
    sum_squared = K.sum(K.square(x-y), axis = 1, keepdims= True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Make embedding
embedding is each sub, identical model

In [8]:
def make_embedding():
    inputs = tf.keras.Input(shape=INPUT_SHAPE)
    base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        input_shape=INPUT_SHAPE,
        include_top=False,
        weights="imagenet"
    )

    # Only make the last few layers (0.1) trainable 
    base_model.trainable = True 
    limit = len(base_model.layers) - int(len(base_model.layers)*0.10)
    for layer in base_model.layers[:limit]:
        base_model.trainable = False

    # create output 
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(64)(x)
    model = tf.keras.Model(inputs, outputs)

    return model
    

# Build siamese model

In [9]:
def make_siamese_model():
    # create embedding
    embedding = make_embedding()

    # create the same embedding for the two inputs 
    input_a = Input(shape = INPUT_SHAPE, name = "first_image")
    input_b = Input(shape = INPUT_SHAPE, name = "second_image")

    embedding_a = embedding(input_a)
    embedding_b = embedding(input_b)

    # Create the final euclidean distance layer
    output = Lambda(euclidean_distance, name = "distance")([embedding_a, embedding_b])

    return Model([input_a, input_b], output)

In [10]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 first_image (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 second_image (InputLayer)      [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 64)           2339968     ['first_image[0][0]',            
                                                                  'second_image[0][0]']     

# Contrastive loss

In [11]:
def contrastive_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)   # cast the 1 and 0 to float32
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(MARGIN-y_pred, 0))
    return y_true*square_pred + (1-y_true)*margin_square

# Compile and train model

In [12]:
class Custom_Precision(tf.keras.metrics.Precision):

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_fix = tf.math.less(y_pred, 0.5) 
        y_pred_fix = tf.cast(y_pred_fix, y_pred.dtype)
       
        return super().update_state(y_true, y_pred_fix, sample_weight)
  
class Custom_Recall(tf.keras.metrics.Recall):

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_fix = tf.math.less(y_pred, 0.5) 
        y_pred_fix = tf.cast(y_pred_fix, y_pred.dtype)
       
        return super().update_state(y_true, y_pred_fix, sample_weight)
  
class Custom_Accuracy(tf.keras.metrics.Accuracy):

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_fix = tf.math.less(y_pred, 0.5) 
        y_pred_fix = tf.cast(y_pred_fix, y_pred.dtype)
       
        return super().update_state(y_true, y_pred_fix, sample_weight)

In [13]:
EPOCHS = 50 

siamese_model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.RMSprop(),metrics=[Custom_Accuracy(), Custom_Precision(), Custom_Recall()])
history = siamese_model.fit(train_dataset, 
                            steps_per_epoch=math.ceil(len(train_pairs) / BATCH_SIZE_TRAIN), 
                            validation_data=validation_dataset, 
                            epochs = EPOCHS
                            )

Epoch 1/50


2025-02-10 14:12:22.535237: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2025-02-10 14:12:22.925508: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-02-10 14:12:22.925965: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-02-10 14:12:22.925972: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2025-02-10 14:12:22.926287: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-02-10 14:12:22.926309: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


  3/107 [..............................] - ETA: 2s - loss: 29.8954 - accuracy: 0.4583 - custom__precision: 0.0000e+00 - custom__recall: 0.0000e+00  

2025-02-10 14:12:23.194180: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


107/107 [==============================] - 12s 80ms/step - loss: 8.0060 - accuracy: 0.5376 - custom__precision: 0.0000e+00 - custom__recall: 0.0000e+00 - val_loss: 3.2473 - val_accuracy: 0.5383 - val_custom__precision: 0.0000e+00 - val_custom__recall: 0.0000e+00
Epoch 2/50
107/107 [==============================] - 8s 75ms/step - loss: 1.8993 - accuracy: 0.5376 - custom__precision: 0.0000e+00 - custom__recall: 0.0000e+00 - val_loss: 1.9035 - val_accuracy: 0.5383 - val_custom__precision: 0.0000e+00 - val_custom__recall: 0.0000e+00
Epoch 3/50
107/107 [==============================] - 8s 75ms/step - loss: 1.3849 - accuracy: 0.5376 - custom__precision: 0.0000e+00 - custom__recall: 0.0000e+00 - val_loss: 1.5679 - val_accuracy: 0.5383 - val_custom__precision: 0.0000e+00 - val_custom__recall: 0.0000e+00
Epoch 4/50
107/107 [==============================] - 8s 74ms/step - loss: 1.3117 - accuracy: 0.5376 - custom__precision: 0.0000e+00 - custom__recall: 0.0000e+00 - val_loss: 1.4400 - val_accu